In [1]:
!pip install playwright


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 15.6 MB/s eta 0:00:00


In [2]:
!playwright install-deps


Installing dependencies...
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,701 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 P

In [3]:
!playwright install chromium


(node:2775) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
167.3 MiB [] 0% 0.0s167.3 MiB [] 0% 62.2s167.3 MiB [] 0% 175.8s167.3 MiB [] 0% 118.3s167.3 MiB [] 0% 70.2s167.3 MiB [] 0% 55.3s167.3 MiB [] 0% 55.0s167.3 MiB [] 0% 53.8s167.3 MiB [] 0% 49.7s167.3 MiB [] 0% 46.3s167.3 MiB [] 0% 40.0s167.3 MiB [] 0% 29.4s167.3 MiB [] 0% 26.5s167.3 MiB [] 1% 23.6s167.3 MiB [] 1% 22.1s167.3 MiB [] 1% 19.9s167.3 MiB [] 1% 18.2s167.3 MiB [] 1% 17.9s167.3 MiB [] 2% 16.8s167.3 MiB [] 2% 16.5s167.3 MiB [] 2% 16.3s167.3 MiB [] 2% 16.9s167.3 MiB [] 2% 14.4s167.3 MiB [] 3% 13.0s167.3 MiB [] 3% 12.1s167.3 MiB [] 4% 11.6s167.3 MiB [] 4% 11.4s167.3 MiB [] 4% 10.8s167.3 MiB [] 4% 11.0s167.3 MiB [] 4% 11.2s167.3 MiB [] 4% 11.9s167.3 MiB [] 4% 12.4s167.3 MiB [] 5% 12.4s167.3 

In [11]:
import json
import re
from urllib.parse import urljoin, quote_plus
from playwright.async_api import async_playwright


AMAZON_BASE = "https://www.amazon.de"
QUERY = "Harry Potter Buch"


async def click_if_present(page, selectors, timeout=2000):
    """Click the first selector that exists and is visible. If none match, do nothing."""
    for selector in selectors:
        try:
            element = page.locator(selector).first
            if await element.count() > 0 and await element.is_visible():
                await element.click(timeout=timeout)
                return True
        except Exception:
            continue
    return False


async def handle_popups(page):
    """Best-effort handling for cookie banners and simple overlays."""
    await click_if_present(
        page,
        selectors=[
            "#sp-cc-accept",
            "text=Alle akzeptieren",
            "text=Akzeptieren",
        ],
    )

    await click_if_present(
        page,
        selectors=[
            "button[aria-label='Schließen']",
            "button[aria-label='Close']",
        ],
    )


async def is_sponsored(card) -> bool:
    """Try to detect sponsored results without parsing the full card text."""
    if await card.locator("text=Gesponsert").count() > 0:
        return True
    if await card.locator("text=Sponsored").count() > 0:
        return True
    if await card.locator('span[data-component-type="s-sponsored-label-info-icon"]').count() > 0:
        return True
    return False


async def extract_title(page) -> str:
    """Extract and normalize the product title."""
    await page.wait_for_selector("#productTitle", timeout=20000)
    raw = (await page.locator("#productTitle").text_content() or "").strip()
    return re.sub(r"\s+", " ", raw)


async def extract_price(page) -> str:
    """Extract a visible price string from common Amazon layouts."""
    price_selectors = [
        "span.a-price span.a-offscreen",
        "span.apexPriceToPay span.a-offscreen",
        "span.reinventPricePriceToPayMargin span.a-offscreen",
        "#priceblock_ourprice",
        "#priceblock_dealprice",
        "#price_inside_buybox",
    ]

    for selector in price_selectors:
        price_node = page.locator(selector).first
        if await price_node.count() > 0:
            text = (await price_node.text_content() or "").strip()
            if text:
                return re.sub(r"\s+", " ", text)

    # Fallback: some pages split price into whole + fraction
    whole = page.locator("span.a-price-whole").first
    fraction = page.locator("span.a-price-fraction").first

    if await whole.count() > 0 and await fraction.count() > 0:
        whole_digits = re.sub(r"[^\d]", "", (await whole.text_content() or ""))
        frac_digits = re.sub(r"[^\d]", "", (await fraction.text_content() or ""))
        if whole_digits and frac_digits:
            return f"€{whole_digits},{frac_digits}"

    return ""


async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)

        context = await browser.new_context(
            locale="de-DE",
            viewport={"width": 1280, "height": 800},
            user_agent=(
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/120.0.0.0 Safari/537.36"
            ),
        )

        page = await context.new_page()

        # 1) Go straight to the search results page (more reliable in Colab than the homepage)
        search_url = f"{AMAZON_BASE}/s?k={quote_plus(QUERY)}"
        await page.goto(search_url, wait_until="domcontentloaded")
        await handle_popups(page)

        # 2) Wait for results to appear (results sometimes load a bit late)
        cards = page.locator('div[data-component-type="s-search-result"]')
        for _ in range(10):
            if await cards.count() > 0:
                break
            await page.wait_for_timeout(500)

        count = await cards.count()
        print("Result cards found:", count)

        # 3) Find the first product URL (prefer non-sponsored; fall back if needed)
        product_url = None

        if count == 0:
            links = page.locator('a[href*="/dp/"], a[href*="/gp/"]')
            if await links.count() == 0:
                raise RuntimeError("No results found (possible block/interstitial).")

            href = await links.first.get_attribute("href")
            product_url = urljoin(AMAZON_BASE, href)
        else:
            fallback_url = None

            for i in range(min(count, 40)):
                card = cards.nth(i)

                link = card.locator('a[href*="/dp/"], a[href*="/gp/"]').first
                if await link.count() == 0:
                    continue

                href = await link.get_attribute("href")
                if not href:
                    continue

                full_url = urljoin(AMAZON_BASE, href)

                if fallback_url is None:
                    fallback_url = full_url

                if await is_sponsored(card):
                    continue

                product_url = full_url
                break

            if product_url is None:
                product_url = fallback_url

            if not product_url:
                raise RuntimeError("Could not extract a product URL from search results.")

        # 4) Open the product page
        await page.goto(product_url, wait_until="domcontentloaded")
        await handle_popups(page)

        # 5) Extract title and price
        title = await extract_title(page)
        price = await extract_price(page)

        # 6) Output JSON
        print(json.dumps({"title": title, "price": price}, ensure_ascii=False))

        await context.close()
        await browser.close()


await main()


Result cards found: 16
{"title": "Harry Potter - Gesamtausgabe (Harry Potter): Ungekürzter Sammelband mit allen 7 Büchern – Reduzierter Preis, nur solange der Vorrat reicht", "price": "€109,00"}
